## Прогнозирование стоимости автомобиля по характеристикам


In [ ]:
# %pip install catboost
# %pip install sklearn
# %pip install tqdm
# %pip install phik
# %pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# %pip install seaborn
# %pip install pandas-profiling[notebook]

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
# from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.preprocessing import LabelEncoder
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
# %pip freeze > requirements.txt
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
VERSION    = 2
DIR_TRAIN  = './input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = './input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%    

# train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
train_my = pd.read_csv('dftrain.csv') # мой датасет для обучения модели



Python       : 3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]
Numpy        : 1.22.0


In [ ]:
# %pip uninstall pandas-profiling
# %pip install pandas-profiling[notebook,html]
# %pip install ruamel-yaml

#использование профилировщика пандас для изучения тестового датасета
# from pandas_profiling import ProfileReport
# profile = ProfileReport(test, title="Pandas Profiling Report")
# profile.to_file("your_report.html")


## Data Preprocessing

In [17]:
# train.dropna(subset=['productionDate','mileage'], inplace=True)
# train.dropna(subset=['price'], inplace=True)
# # для baseline просто возьму пару схожих признаков без полной обработки
# columns = ['bodyType', 'brand', 'productionDate', 'engineDisplacement', 'mileage']

# df_train = train[columns]
# df_test = test[columns]
# y = train['price']

In [2]:
#из-за нехватки времени некоторые признаки в тренировочной базе являются неполными
# адаптировать тестовый датасет, чтобы там не было отсутствующих значений для  таких признаков:
mycolors=['чёрный', 'белый', 'серый', 'синий', 'серебристый','красный']
test['color']=test['color'].apply(lambda x: x if (x in mycolors) else 'другой цвет')
test['fuelType']=test['fuelType'].apply(lambda x: x if (x in ['бензин','дизель','гибрид']) else 'другое топливо')

train_my['numberOfDoors']=train_my['numberOfDoors'].apply(lambda x: int(x))


In [3]:
# 'complectation_dict','equipment_dict','model_info','super_gen',
columns_my=['bodyType', 'brand', 'color', 
        'engineDisplacement', 'enginePower', 
       'fuelType',  'mileage', 'modelDate',  'model_name',
       'name', 'numberOfDoors',  
       'productionDate', 'sell_id',  'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
       'Привод', 'Руль']
# columns_my = ['brand', 'color','bodyType' , #'productionDate', 'engineDisplacement', 'mileage',
#        'fuelType'#,'modelDate'
# ]

# , 'complectation_dict'
# , 'equipment_dict',
# 'mileage',
#  'model_info',
# 'fuelType',
# 'parsing_unixtime',
# 'sell_id',
# 'super_gen','modelDate','productionDate', 
cat_features_ids=['bodyType', 'brand', 'color',
        'engineDisplacement', 'enginePower',
           'model_name','name','fuelType',
        'numberOfDoors',  
         'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы',  'ПТС',
       'Привод', 'Руль']

# обновить признаки категорий в соответствии с выбранными столбцами из датасета
# (при итеративном отключении и включении разных столбцов в тренировочные признаки при попытке выяснить, какой столбец рушит моментально catboost)
cat_features_ids=list(set(columns_my).intersection(set(cat_features_ids)))




In [4]:
# выбрать столбцы для передачи в регрессоры
df_train_my = train_my[columns_my]
y = train_my['price']
df_test = test[columns_my]

df_train_my['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
data_my = df_test.append(df_train_my, sort=False).reset_index(drop=True) # объединяем


C:\Users\User\AppData\Local\Temp/ipykernel_4800/1997197722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_my['sample'] = 1 # помечаем где у нас трейн
C:\Users\User\AppData\Local\Temp/ipykernel_4800/1997197722.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sample'] = 0 # помечаем где у нас тест


## Train Split

In [76]:
print()

{'engineDisplacement', 'bodyType', 'model_name', 'brand', 'productionDate', 'modelDate', 'color', 'vehicleConfiguration', 'vendor', 'enginePower', 'Владельцы', 'mileage', 'Привод', 'vehicleTransmission', 'ПТС', 'Руль', 'sample', 'fuelType', 'name', 'numberOfDoors', 'sell_id'}


# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

class AlgRegressor:
    algs={}
    cat_features=None,None
    isFit=False
    lastbestparams=None
    rf_random=None
    def __init__(self,data_my,y,cat_features=None) -> None:
        self.y=y
        if cat_features is None:
            pass
        else:
            self.cat_features=cat_features
            for colum in cat_features:
                data_my[colum] = data_my[colum].astype('category').cat.codes
        scaler = StandardScaler()
        sample=set(data_my.columns).difference('sample')
        data_my_ = pd.DataFrame(data=scaler.fit_transform(data_my), columns=sample)
        data_my_['sample']=data_my['sample']
        # приготовить скалированные данные и
        self.X_ = data_my_.query('sample == 1').drop(['sample'], axis=1)
        self.X_sub_ = data_my_.query('sample == 0').drop(['sample'], axis=1)
        # и неизменные
        self.X = data_my.query('sample == 1').drop(['sample'], axis=1)
        self.X_sub = data_my.query('sample == 0').drop(['sample'], axis=1)
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)       
        self.X_train_, self.X_test_, self.y_train_, self.y_test_ = train_test_split(self.X_, self.y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)       
        super().__init__()

    def addalg(self,_name_,alg):
        self.algs[_name_]=alg
        print(_name_)
    def fitpredict(self,algname,isScaler=False,isX_sub=False):
        # if not (X_sub is None) and not self.isFit: raise Exception('не было обучения')
        alg=self.algs[algname]
        if isX_sub: 
            if isScaler:
                return alg.predict(self.X_sub_)
            else:
                return alg.predict(self.X_sub)
        else:
            if isScaler:
                alg.fit(self.X_train_, self.y_train_)
                return alg.predict(self.X_test_)
            else:
                alg.fit(self.X_train, self.y_train)
                # self.isFit=True
                return alg.predict(self.X_test)
    def setp(self,algname,**params):
        self.algs[algname].set_params(**params)
        for qq in params:
            print(str(self.algs[algname].get_params()[qq]))
    def fullname(algname):
        return str(alg.algs[algname]).split(".")[-1].rstrip("'>")

    def SearchCV(self,algname,isScaler=False,rf_random=None):
        if (self.algs[algname] is None): raise Exception('не инициализирован регрессор')
        if rf_random is None:
            n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
            max_features = ['auto', 'sqrt']
            max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
            max_depth.append(None)
            min_samples_split = [2, 5, 10]
            min_samples_leaf = [1, 2, 4]
            bootstrap = [True, False]
            random_grid = {'n_estimators': n_estimators,
                        'max_features': max_features,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'bootstrap': bootstrap}
            self.searchCV= RandomizedSearchCV(estimator=self.algs[algname], param_distributions=random_grid, n_iter=100, 
                                cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
        else: self.searchCV=rf_random
        
        
        if isScaler:
            self.searchCV.fit(self.X_train, self.y_train)                    
        else:
            self.searchCV.fit(self.X_train_, self.y_train_)  
        self.lastbestparams=rf_random.best_params_
        self.algs[algname].set_params(**self.lastbestparams)
    # здесь передаются ndarrays , а не dataframe
    def compute_meta_feature(self,clf, X_train, X_test, y_train, cv):
        """
        Computes meta-features using the regressor.
        :arg clf: scikit-learn regressor
        :args X_train, y_train: training set
        :arg X_test: testing set
        :arg cv: cross-validation folding
        """
        X_meta_train = np.zeros_like(y_train, dtype=np.float32)
        for train_fold_index, predict_fold_index in cv.split(X_train):
            X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
            y_fold_train = y_train[train_fold_index]
            
            folded_clf = clone(clf)
            folded_clf.fit(X_fold_train, y_fold_train)
            X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
        
        meta_clf = clone(clf)
        meta_clf.fit(X_train, y_train)
        
        X_meta_test = meta_clf.predict(X_test)
        
        return X_meta_train, X_meta_test
    def generate_meta_features(self,nregressors,  cv,isScaler=False):
        """
        Generates metafeatures using a list of regressors.
        :arg regressors: list of scikit-learn regressors
        :args X_train, y_train: training set
        :arg X_test: testing set
        :arg cv: cross-validation folding
        """
        if isScaler:
            X_train, X_test, y_train=alg.X_train_.to_numpy(), alg.X_test_.to_numpy(), alg.y_train_.to_numpy()
        else:
            X_train, X_test, y_train=alg.X_train.to_numpy(), alg.X_test.to_numpy(), alg.y_train_.to_numpy()
        regressors=[alg.algs[i] for i in nregressors]
        features = [
            compute_meta_feature(clf, X_train, X_test, y_train, cv)
            for clf in tqdm(regressors)
        ]
        
        tr = np.vstack([
            features_train for features_train, features_test in features
        ]).T

        tst = np.vstack([
            features_test for features_train, features_test in features
        ]).T
        if isScaler:
            self.stacked_features_train_=tr
            self.stacked_features_test_= tst
        else:
            self.stacked_features_train=tr
            self.stacked_features_test=tst
        
        return tr, tst



In [6]:
alg=AlgRegressor(data_my,y,cat_features_ids)
# добавление разных регрессоров
alg.addalg('LR',LinearRegression())
alg.addalg('DTR',DecisionTreeRegressor(random_state=RANDOM_SEED))
alg.addalg('RFR',RandomForestRegressor(random_state=RANDOM_SEED))
alg.addalg('Ada',AdaBoostRegressor(random_state=RANDOM_SEED))
alg.addalg('GBoo',GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, 
                max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1, max_features=int(len(data_my.columns)/3), random_state=RANDOM_SEED))

DTR
RFR
Ada
GBoo


In [7]:
def predictionresult(algname): #проверка для выбранного алгоритма на немасштабированных и масштабированных признаках
    predict= (alg.fitpredict(algname),alg.fitpredict(algname,isScaler=True))
    print("Точность {}/(StandardScaler) по метрике MAPE,%: {:0.2f}/{:0.2f}".format(alg.fullname(algname),
    mean_absolute_percentage_error(alg.y_test_, predict[0])*100,
    mean_absolute_percentage_error(alg.y_test, predict[1])*100))

LR


In [8]:
# линейная регрессия хуже остальных алгоритмов предсказывает
predictionresult('LR')

Точность LinearRegression()/(StandardScaler) по метрике MAPE,%: 83.87/85.43


In [135]:
predictionresult('Ada')

Точность AdaBoostRegressor/(StandardScaler) по метрике MAPE,%: 439.98/436.39


In [156]:
predictionresult('GBoo')

Точность GradientBoostingRegressor(max_features=7, random_state=42, subsample=1/(StandardScaler) по метрике MAPE,%: 27.93/28.41


In [158]:
bparams={'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20}
alg.setp('GBoo',**bparams)
predictionresult('GBoo')


1200
10
4
sqrt
20
Точность GradientBoostingRegressor(max_depth=20, max_features='sqrt', min_samples_leaf=4,
                          min_samples_split=10, n_estimators=1200,
                          random_state=42, subsample=1)/(StandardScaler) по метрике MAPE,%: 18.93/19.39


In [159]:
predictionresult('DTR')

Точность DecisionTreeRegressor(max_depth=4, random_state=42)/(StandardScaler) по метрике MAPE,%: 45.57/45.57


In [160]:
predictionresult('RFR') #точность RandomForestRegressor по MAPE из коробки получилась выше, чем с подбором гиперпараметров

Точность RandomForestRegressor(random_state=42)/(StandardScaler) по метрике MAPE,%: 18.93/19.08


In [161]:
# всю ночь выполнялся перебор alg.SearchCV('RFR')
# но лучших результатов не получилось
# Fitting 3 folds for each of 100 candidates, totalling 300 fits
# {'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
bparams={'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}
alg.setp('RFR',**bparams)
predictionresult('RFR')

1200
10
4
sqrt
20
False
Точность RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_leaf=4, min_samples_split=10,
                      n_estimators=1200, random_state=42)/(StandardScaler) по метрике MAPE,%: 19.33/19.27


In [ ]:
# 23.20% при max_depth=14
# быстрее на  порядок считает, чем RandomForestRegressor
for depth in range(4,25):
    alg.setp('DTR',max_depth=depth)
    predict= alg.fitpredict('DTR',isScaler=True)
    print(f"Точность DecisionTreeRegressor по метрике MAPE: {(mape(alg.y_test_, predict))*100:0.2f}%")

In [ ]:
# 22.99% при max_depth=14
for depth in range(4,25):
    alg.setp('DTR',max_depth=depth)
    predict= alg.fitpredict('DTR')
    print(f"Точность DecisionTreeRegressor по метрике MAPE: {(mape(alg.y_test, predict))*100:0.2f}%")

In [30]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the regressor.
    :arg clf: scikit-learn regressor
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict(X_test)
    
    return X_meta_train, X_meta_test
def generate_meta_features(regressors, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of regressors.
    :arg regressors: list of scikit-learn regressors
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(regressors)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

cv = KFold(n_splits=10, shuffle=True, random_state=42)


def fitfit(clf, X_train, y_train, X_test,y_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    print("Точность {}/(StandardScaler) по метрике MAPE,%: {:0.2f}/{:0.2f}".format('стекинг',
    mean_absolute_percentage_error(y_test, y_test_pred)*100,0))
    return y_test_pred


In [32]:
nrgs=['DTR','LR','RFR','GBoo']
rgs=[alg.algs[i] for i in nrgs]

qq=[alg.algs['DTR'],
    alg.algs['LR'],
    alg.algs['RFR'],
    alg.algs['GBoo']]
print(rgs==qq)

True


In [ ]:
nalgs=['DTR','LR','RFR','GBoo']
stacked_features_train_, stacked_features_test_ = alg.generate_meta_features(nalgs, cv,isScaler=True)    
for Alg in nalgs:
    clf = alg.algs[Alg]
    clf.fit(stacked_features_train_, alg.y_train_)

stacked_features_train, stacked_features_test = alg.generate_meta_features(nalgs, cv,isScaler=False)    

y_test_pred=fitfit(clf, X_train=stacked_features_train,y_train=alg.y_train_, X_test=stacked_features_test,y_test=alg.y_test_)

print(y_test_pred)

# # Model 2 : CatBoost


## Fit

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
# попытки разобраться с признаком parse_unixtime
# from datetime import datetime
# def parseunix(x):
#     ts = int(x)
#     return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

# df_=test.loc[:,['parsing_unixtime']] 
# df_['unixtime']=df_['parsing_unixtime'].apply(parseunix)

# df_.sort_values(by='unixtime',ascending=True)

In [13]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=20,
         use_best_model=True,
        #  plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [14]:
predict_test = np.exp(model.predict(X_test))
predict_submission = np.exp(model.predict(X_sub))

In [15]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 19.33%


Как видим точность возросла до 15%, а что будет на ЛБ?

# Submission

In [17]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_myds_v{VERSION}.csv', index=False)
sample_submission.head(10)
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   sell_id  34686 non-null  int64  
 1   price    34686 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 542.1 KB


В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing